<a href="https://colab.research.google.com/github/Nikhil-Nandam/TensorFlow-Notebooks/blob/main/08_Introduction_to_NLP_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences, text or speech).

Another common term for NLP Problems is sequence to sequence problems (seq2seq).

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import a series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-06-07 13:03:57--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-06-07 13:03:57 (65.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster).

See the [original source](https://www.kaggle.com/competitions/nlp-getting-started).

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-06-07 13:04:05--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.26.128, 172.217.193.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.26.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-06-07 13:04:05 (93.4 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would to us Python.

But it is better to visualize it right away.

So, another way to do this would be through pandas...

In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
# Shuffle training dataset
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class are there?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [20]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)   # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index + 5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 1 (real disaster)
Text:
Two trains have collided in India. Please pray for victims their families and rescuers.

---

Target: 0 (not real disaster)
Text:
@questergirl ditto but its all we had. and the way i feel if i drank vodka over ice they would have to body bag me

---

Target: 0 (not real disaster)
Text:
#Turkish couple decided to feed 4000 #Syrian #refugees as part of their #wedding celebrations http://t.co/EHLq3ZSPTd http://t.co/DjX5eLbrv1

---

Target: 1 (real disaster)
Text:
There's a sinkhole in Brooklyn ?!

---

Target: 1 (real disaster)
Text:
A STRONG THUNDERSTORM WILL AFFECT CENTRAL HALIFAX COUNTY THROUGH 1145 PM EDT for Halifax [VA] till 11:45 PM EDT http://t.co/MjSTefgGU5

---



### Split data into training and validation sets

In [22]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split the training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(), 
                                                                            train_df_shuffled["target"].to_numpy(), 
                                                                            random_state=42, 
                                                                            test_size=0.1)    # use 10% of training data for validation split

In [23]:
# Check the lengths 
len(train_sentences), len(train_labels), len(val_sentences), len (val_labels)

(6851, 6851, 762, 762)

In [24]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text Vectorization (Tokenization)

In [28]:
from pandas.core.common import standardize_mapping
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None,    # how many words in the vocabulary (automatically add <OOV> (Out of Vocabulary))
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,        # create groups of n-words
                                    output_mode="int",
                                    output_sequence_length=None,    # how long do you want your sequences to be
                                    pad_to_max_tokens=False)

In [30]:
len(train_sentences[0].split())

7

In [32]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [33]:
# Setup text vectorization variables
max_vocab_length = 10000    # max number of words to have in our vocabulary
max_length = 15             # max length our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [35]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [37]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [42]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 'Every kingdom divided against itself is headed for destruction and no city or house divided against itself will stand.'
Matthew 12:32        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 197, 3703, 5798,  445, 2418,    9, 1843,   10,  399,    7,   40,
         182,   53,  318, 5798]])>

In [43]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()   # get all of the unique words in the training data
top_5_words = words_in_vocab[:5]        # get the most common words
bottom_5_words = words_in_vocab[-5:]    # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use [TensorFlow's Embedding layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)

The parameters we care the most about for our embedding layer:
* `input_dim`: the size of our vocabulary
* `output_dim`: the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length`: length of the sequences being passed to the embedding layer

In [54]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim=max_vocab_length,   # set the input shape
                      output_dim=128,               # output shape
                      embeddings_initializer="uniform",
                      input_length=max_length)      # how long is each input

embedding        

In [57]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Accident in #Ashville on US 23 SB before SR 752 #traffic http://t.co/hylMo0WgFI        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01656544,  0.04472256, -0.00726734, ..., -0.0424385 ,
          0.02992496, -0.04187266],
        [ 0.02136407, -0.00486449, -0.01888036, ..., -0.03720061,
          0.00889454,  0.02111851],
        [-0.00439882,  0.03087072, -0.00621402, ..., -0.01486688,
         -0.00074391,  0.00197939],
        ...,
        [ 0.0422004 ,  0.04687983, -0.00243538, ...,  0.03409253,
         -0.04226196, -0.02294757],
        [ 0.0422004 ,  0.04687983, -0.00243538, ...,  0.03409253,
         -0.04226196, -0.02294757],
        [ 0.0422004 ,  0.04687983, -0.00243538, ...,  0.03409253,
         -0.04226196, -0.02294757]]], dtype=float32)>

In [59]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-1.65654421e-02,  4.47225608e-02, -7.26734474e-03, -4.64196578e-02,
         2.28560083e-02,  2.62981765e-02, -2.92411577e-02, -2.49337684e-02,
         3.87248062e-02,  1.19741559e-02, -1.54986382e-02,  4.47545908e-02,
        -2.66950969e-02, -2.67418157e-02,  1.97914280e-02, -5.91993332e-04,
        -1.47470012e-02,  1.80922858e-02,  4.38948385e-02,  4.51011322e-02,
         2.48231031e-02,  1.51066668e-02,  3.98152135e-02,  3.81464250e-02,
         6.70958683e-03,  1.20772012e-02,  4.97720577e-02,  8.73195007e-03,
         4.84531857e-02,  3.43615524e-02, -3.67099531e-02,  1.55790113e-02,
         2.43876316e-02, -4.47317213e-03, -3.78604159e-02,  2.07022317e-02,
         1.81962587e-02,  2.56689675e-02, -2.51942165e-02, -1.98456291e-02,
         3.41677330e-02, -3.21323760e-02,  3.88959795e-03, -2.13150140e-02,
         2.74461620e-02, -5.12138754e-03,  3.18524353e-02, -4.03685793e-02,
         1.92789547e-02, -3.18403244e-0

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.
* Model 0: Naïve Bayes (baseline), this is from [Sklearn ML map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with TensorFlow
* Create a model
* Build the model
* Fit the model
* Evaluate the model

### Model 0: Getting a baseline

As with all Machine Learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll be using Sklearn's Multinomial Naïïve Bayes using the TF_IDF formula to convert our words to numbers.

> 🔑**Note:** It's common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),   # convert words to numbers using tfidf
    ("clf", MultinomialNB()),       # model the text
])

# Fit the pipeline to training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [63]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score * 100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [67]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])